### Installing Requirements

In [1]:
from  google.cloud import bigquery
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

# Reach & Frequency Analysis

In [2]:
client = bigquery.Client(project='mightyhive-data-science-poc')

/Users/marianoarouxet/Desktop/projects/adh_norton/env/lib/python3.7/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Load data from bigquery

In [3]:
query = "SELECT * FROM `adh-connection.adh_results.ma_freqAnalysis_DV360_LifeLock_Rem_202202_202205` LIMIT 1000"
df = client.query(query)
df= df.to_dataframe()
# Drop first row (results that have privacy issues)
df = df[1:]
# Cost is in a different scale
df['total_cost'] = np.round(df['total_cost']/1000000000,2)
df.head()

,advertiser_id,campaign_id,frequency,reach,total_impressions,total_clicks,total_conversions,total_revenue,total_cost
1,4696533,4446501,1,35356,35356,254,74,11234.27,779.76
2,4696533,4446501,2,29798,59596,287,94,12894.71,1155.37
3,4696533,4446501,3,19779,59337,266,57,7959.73,1230.84
4,4696533,4446501,4,19360,77440,318,58,8734.49,1456.62
5,4696533,4446501,5,14582,72910,249,37,4688.17,1463.76


### Global metrics

In [5]:
# Create a DataFrame with global metrics, rename index and columns
resume = pd.DataFrame(
    df[['reach','total_impressions','total_clicks','total_conversions','total_revenue','total_cost']].sum().apply(int)
).transpose().rename(index={0:'ADH Metrics'},columns={'reach':'Reach','total_impressions':'Impressions','total_clicks':'Clicks','total_conversions':'Conversions','total_revenue':'Revenue [$]','total_cost':'Cost [$]'})

resume

,Reach,Impressions,Clicks,Conversions,Revenue [$],Cost [$]
ADH Metrics,932787,59847715,65153,1262,156309,866365


In [6]:
# Read CM report from CSV and format it
cm_data = pd.read_csv('CM reports/lifelock_globals.csv',skiprows=13)

cm_data.rename(
    columns={
        'DV360 Advertiser ID':'Advertiser Id', 
        'DV360 Insertion Order ID':'Insertion Order ID', 
        'Total Conversions': 'Conversions', 
        'DV360 Cost USD':'Cost [$]', 
        'Total Revenue' : 'Revenue [$]'
        },
        inplace=True
)
cm_data.set_index('Insertion Order ID', inplace=True)
cm_data.rename(index={'14747641':'CM Metrics'}, inplace=True)
cm_data = pd.DataFrame(cm_data.loc['CM Metrics',['Impressions','Clicks','Conversions','Cost [$]','Revenue [$]']].apply(int)).transpose()
cm_data

,Impressions,Clicks,Conversions,Cost [$],Revenue [$]
CM Metrics,58699606,68085,1221,857080,151111


In [7]:
# Concat both dataframes and add CPA, CR and ROAS columns 
resume = pd.concat([resume,cm_data], axis = 0)
resume['CPA [$]'] =  np.round(resume['Cost [$]']/resume['Conversions'],2)
resume['CTR [%]'] =  np.round(resume['Clicks']/resume['Impressions'],6) * 100
resume['Conversion Rate [%]'] =  np.round(resume['Conversions']/resume['Impressions'],6) * 100
resume['ROAS [%]'] =  np.round(resume['Revenue [$]']/resume['Cost [$]'],4) * 100
resume


,Reach,Impressions,Clicks,Conversions,Revenue [$],Cost [$],CPA [$],CTR [%],Conversion Rate [%],ROAS [%]
ADH Metrics,932787.0,59847715,65153,1262,156309,866365,686.50,0.1089,0.0021,18.04
CM Metrics,NaN,58699606,68085,1221,151111,857080,701.95,0.1160,0.0021,17.63


### Charts configuration

In [6]:
# Colors for plots
c_reach = '#ff007f'
c_impressions = '#F5A400'
c_clicks = '#005daa'
c_conversions = '#1CAC18'
c_revenue = '#b15a8b'
c_cost = '#415a6c'
# min and max frequency
min_f = 0
max_f = 20

### Analysis without grouping frequency in intervals (not very useful for Lifelock REM)

In [10]:
# Create acumulative variables
# Get all the columns except advertiser_id, campaign_id, frequency
dimensions = [  
            'reach',
            'total_impressions',
            'total_clicks',
            'total_conversions',
            'total_revenue',
            'total_cost'
            ]
# For that dimension calculate cumsum and add it as a new column to the dataframe
for column in dimensions:
    df['cumsum_'+column] = df[column].cumsum()
# Preview the dataframe


# Calculate ROAS and CPA
df['CVR'] = df.total_conversions/ df.total_impressions
#df['CPC'] = df.total_cost/ df.total_clicks
df['CPA'] = df.total_cost/ df.total_conversions
#df['CTR'] = df.total_clicks/ df.total_impressions
df['ROAS'] = df.total_revenue / df.total_cost
df['cumsumCVR'] = df.cumsum_total_conversions/ df.cumsum_total_impressions
#df['CPC'] = df.total_cost/ df.total_clicks
df['cumsumCPA'] = df.cumsum_total_cost/ df.cumsum_total_conversions
#df['CTR'] = df.total_clicks/ df.total_impressions
df['cumsumROAS'] = df.cumsum_total_revenue / df.cumsum_total_cost


df.head()

,advertiser_id,campaign_id,frequency,reach,total_impressions,total_clicks,total_conversions,total_revenue,total_cost,cumsum_reach,...,cumsum_total_clicks,cumsum_total_conversions,cumsum_total_revenue,cumsum_total_cost,CVR,CPA,ROAS,cumsumCVR,cumsumCPA,cumsumROAS
1,4696533,4446501,1,35356,35356,254,74,11234.27,779.76,35356,...,254,74,11234.27,779.76,0.002093,10.537297,14.407343,0.002093,10.537297,14.407343
2,4696533,4446501,2,29798,59596,287,94,12894.71,1155.37,65154,...,541,168,24128.98,1935.13,0.001577,12.29117,11.160676,0.001769,11.518631,12.468919
3,4696533,4446501,3,19779,59337,266,57,7959.73,1230.84,84933,...,807,225,32088.71,3165.97,0.000961,21.593684,6.466909,0.001458,14.070978,10.135507
4,4696533,4446501,4,19360,77440,318,58,8734.49,1456.62,104293,...,1125,283,40823.20,4622.59,0.000749,25.114138,5.996409,0.001221,16.33424,8.83124
5,4696533,4446501,5,14582,72910,249,37,4688.17,1463.76,118875,...,1374,320,45511.37,6086.35,0.000507,39.561081,3.202827,0.00105,19.019844,7.477613


#### Plot 1 - Share of different metrics and cumulative cost vs frequency

In [19]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df.frequency,
                     y=df["reach"]/df["reach"].sum()*100,
                     name="% of Reach", 
                     marker_color=c_reach
                     ), 
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df.frequency,
                     y=df["total_impressions"]/df["total_impressions"].sum()*100,
                     name="% of Impressions", 
                     marker_color=c_impressions
                     ), 
                     secondary_y=False,)


fig.add_trace(go.Bar(x=df.frequency,
                     y=df["total_clicks"]/df["total_clicks"].sum()*100,
                     name="% of Clicks", 
                     marker_color=c_clicks
                     ), 
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df.frequency,
                     y=df["total_conversions"]/df["total_conversions"].sum()*100,
                     name=" % of Conversion", 
                     marker_color=c_conversions
                     ), 
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df.frequency,
                     y=df["total_revenue"]/df["total_revenue"].sum()*100,
                     name=" % of Revenue", 
                     marker_color=c_revenue
                     ), 
                     secondary_y=False,)
fig.add_trace(go.Scatter(

                        x=df.frequency, 
                        y=df.cumsum_total_cost,
                        mode='lines+markers',
                        name='Cost', 
                        marker_color=c_cost
                        ), 
                        secondary_y=True,)


fig.update_layout(
    title='<b>Share of different metrics and cumulative cost vs frequency</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        tickvals = df["frequency"],
        ticktext = df["frequency"]),
    xaxis_title="<b>Frequency</b>",
    xaxis_range=[0,100]
)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>% of metric</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>USD Dollar</b>", secondary_y=True)

#### Plot 2 - CPA And ROAS vs Frequency

In [20]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df.frequency, 
                         y=df.cumsumCPA,
                         mode='lines',
                         name='CPA', marker_color='#ff007f'),
                         secondary_y=False,)

fig.add_trace(go.Scatter(x=df.frequency, 
                         y=df.cumsumROAS,
                         mode='lines',
                         name='ROAS', marker_color='#415a6c'),
                         secondary_y=True,)

fig.update_layout(
    title='<b>CPA And ROAS vs Frequency</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        ),
    xaxis_title="<b>Frequency</b>",
    xaxis_range=[0,200]  
    #yaxis2_range = [-1.5,7]    
)


fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>CPA ($) </b>", ticksuffix = "$", secondary_y=False)
fig.update_yaxes(title_text="<b>ROAS</b>", secondary_y=True)

### Analysis in frequency intervals 1 

#### Create the frequency ranges to estimate the metrics per range of frequency. We are going to show the percentages.

In [4]:
df_an = df[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]
df_an = df_an[df_an.frequency > 0]

# Upper limit of each group
#upper_limits = [10,20,30,40,50,75,100,150,200,300,500]
upper_limits = list(range(50,df.frequency.max(),50))

# Create a column called 'frequency_range' to group frequencies based on limits and one numerical column to sort values
df_an.loc[df_an['frequency']<=upper_limits[0],'frequency_range'] = f'1 - {upper_limits[0]}'
df_an.loc[df_an['frequency']<=upper_limits[0],'order'] = 0
for index,value in enumerate(upper_limits[:-1]):
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'frequency_range'] = f'{upper_limits[index]+1} - {upper_limits[index+1]}'
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'order'] = index+1
df_an.loc[df_an['frequency']>upper_limits[-1],'frequency_range'] = f'>{upper_limits[-1]}'
df_an.loc[df_an['frequency']>upper_limits[-1],'order'] = len(upper_limits)+1
# display updated DataFrame
#df_an = df_an[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']].groupby(by='frequency_range').sum()
df_an = df_an.groupby(by=['order','frequency_range']).sum().reset_index()[['frequency_range','reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]

# Calculate cumulative variables
dimensions = [  
            'reach',
            'total_impressions',
            'total_clicks',
            'total_conversions',
            'total_revenue',
            'total_cost'
            ]
# For that dimension calculate cumsum and add it as a new column to the dataframe
for column in dimensions:
    df_an['cumsum_'+column] = df_an[column].cumsum()


# Calculate CVR and CPA
df_an['CVR'] = df_an.total_conversions/ df_an.total_impressions
df_an['CPA'] = df_an.total_cost/ df_an.total_conversions
df_an['ROAS'] = df_an.total_revenue/ df_an.total_cost
df_an['cumsumCVR'] = df_an.cumsum_total_conversions/ df_an.cumsum_total_impressions
df_an['cumsumCPA'] = df_an.cumsum_total_cost/ df_an.cumsum_total_conversions
df_an['cumsumROAS'] = df_an.cumsum_total_revenue / df_an.cumsum_total_cost

df_an.head()


,frequency_range,reach,total_impressions,total_clicks,total_conversions,total_revenue,total_cost,cumsum_reach,cumsum_total_impressions,cumsum_total_clicks,cumsum_total_conversions,cumsum_total_revenue,cumsum_total_cost,CVR,CPA,ROAS,cumsumCVR,cumsumCPA,cumsumROAS
0,1 - 50,539402,9965783,14815,970,124574.19,177730.98,539402,9965783,14815,970,124574.19,177730.98,0.000097,183.227814,0.700914,0.000097,183.227814,0.700914
1,51 - 100,194022,14262061,13042,143,15622.61,229280.27,733424,24227844,27857,1113,140196.80,407011.25,0.00001,1603.358531,0.068138,0.000046,365.688455,0.344454
2,101 - 150,96571,11708860,10708,65,7152.36,179144.75,829995,35936704,38565,1178,147349.16,586156.0,0.000006,2756.073077,0.039925,0.000033,497.585739,0.251382
3,151 - 200,45708,7917118,8409,42,5856.62,107551.14,875703,43853822,46974,1220,153205.78,693707.14,0.000005,2560.741429,0.054454,0.000028,568.61241,0.220851
4,201 - 250,24621,5482049,5895,18,1127.00,66790.81,900324,49335871,52869,1238,154332.78,760497.95,0.000003,3710.600556,0.016874,0.000025,614.295598,0.202936


#### Plot 3 - Share of different metrics and % of cumulative cost vs frequency bands

In [7]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["reach"]/df_an["cumsum_reach"].max()*100,
                     name="% Reach", marker_color=c_reach), 
                     secondary_y=False)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_impressions"]/df_an["cumsum_total_impressions"].max()*100,
                     name="% Impressions", marker_color=c_impressions),
                     secondary_y=False)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_clicks"]/df_an["cumsum_total_clicks"].max()*100,
                     name="% Clicks", marker_color=c_clicks),
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_conversions"]/df_an["cumsum_total_conversions"].max()*100,
                     name="% Conversions", marker_color=c_conversions),
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_revenue"]/df_an["cumsum_total_revenue"].max()*100,
                     name="% Revenue", marker_color=c_revenue),
                     secondary_y=False,)

fig.add_trace(go.Scatter(x=df_an.frequency_range, 
                         y=df_an.cumsum_total_cost/df_an.cumsum_total_cost.max()*100,
                         mode='lines+markers',
                         name='% of Total cost', marker_color=c_cost),
                         secondary_y=True,)

fig.update_layout(
    title='<b>Share of different metrics and % of cumulative cost vs frequency bands</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        ),
    xaxis_title="<b>Frequency</b>",
    #xaxis_range=[0,10]    
)


fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>Percentage</b>", ticksuffix = "%", secondary_y=False)
fig.update_yaxes(title_text="<b>Cost</b>", ticksuffix = "%", secondary_y=True)

####  Plot 4 - Share of different metrics and % cost vs frequency band

In [8]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["reach"]/df_an["cumsum_reach"].max()*100,
                     name="% Reach", marker_color='#ff007f'), 
                     secondary_y=False)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_impressions"]/df_an["cumsum_total_impressions"].max()*100,
                     name="% Impressions", marker_color='#f0cd28'),
                     secondary_y=False)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_clicks"]/df_an["cumsum_total_clicks"].max()*100,
                     name="% Clicks", marker_color='#29B325'),
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_conversions"]/df_an["cumsum_total_conversions"].max()*100,
                     name="% Conversions", marker_color='#005daa'),
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_revenue"]/df_an["cumsum_total_revenue"].max()*100,
                     name="% Revenue", marker_color='#a87532'),
                     secondary_y=False,)

fig.add_trace(go.Scatter(x=df_an.frequency_range, 
                         y=df_an.total_cost/df_an.cumsum_total_cost.max()*100,
                         mode='lines+markers',
                         name='% of Total cost', marker_color='#415a6c'),
                         secondary_y=True,)


fig.update_layout(
    title='<b>Share of different metrics and % cost vs frequency bands</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        ),
    xaxis_title="<b>Frequency</b>",
    #xaxis_range=[0,10]    
)


fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>Percentage</b>", ticksuffix = "%", secondary_y=False)
fig.update_yaxes(title_text="<b>Cost</b>", ticksuffix = "%", secondary_y=True)

#### Plot 5 - CPA and ROAS vs Frequency band

In [21]:
#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=df_an.frequency_range, 
                         y=df_an.cumsumCPA,
                         mode='lines+markers',
                         name='CPA', marker_color='#ff007f'),
                         secondary_y=False,)

fig.add_trace(go.Scatter(x=df_an.frequency_range, 
                         y=df_an.cumsumROAS,
                         mode='lines+markers',
                         name='ROAS', marker_color='#415a6c'),
                         secondary_y=True,)

fig.update_layout(
    title='<b>CPA and ROAS vs Frequency (calculated using cumulative values)</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        ),
    xaxis_title="<b>Frequency</b>",
    #xaxis_range=[0,40]  
    #yaxis2_range = [-1.5,3]  
)


fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>CPA ($) </b>", ticksuffix = "$", secondary_y=False)
fig.update_yaxes(title_text="<b>ROAS</b>", secondary_y=True)

### Analysis in frequency bands 2

#### Create the frequency ranges to estimate the metrics per range of frequency. We are going to show the percentages.

In [22]:
df_an = df[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]
df_an = df_an[df_an.frequency > 0]

# Upper limit of each group
upper_limits = [5,10,25,50,100,150,200,500]

# Create a column called 'frequency_range' to group frequencies based on limits and one numerical column to sort values
df_an.loc[df_an['frequency']<=upper_limits[0],'frequency_range'] = f'1 - {upper_limits[0]}'
df_an.loc[df_an['frequency']<=upper_limits[0],'order'] = 0
for index,value in enumerate(upper_limits[:-1]):
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'frequency_range'] = f'{upper_limits[index]+1} - {upper_limits[index+1]}'
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'order'] = index+1
df_an.loc[df_an['frequency']>upper_limits[-1],'frequency_range'] = f'>{upper_limits[-1]}'
df_an.loc[df_an['frequency']>upper_limits[-1],'order'] = len(upper_limits)+1
# display updated DataFrame
#df_an = df_an[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']].groupby(by='frequency_range').sum()
df_an = df_an.groupby(by=['order','frequency_range']).sum().reset_index()[['frequency_range','reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]

dimensions = [  
            'reach',
            'total_impressions',
            'total_clicks',
            'total_conversions',
            'total_revenue',
            'total_cost'
            ]
# For that dimension calculate cumsum and add it as a new column to the dataframe
for column in dimensions:
    df_an['cumsum_'+column] = df_an[column].cumsum()

# Calculate CVR and CPA
df_an['CVR'] = df_an.total_conversions/ df_an.total_impressions
df_an['CPA'] = df_an.total_cost/ df_an.total_conversions
df_an['ROAS'] = df_an.total_revenue/ df_an.total_cost
df_an['cumsumCVR'] = df_an.cumsum_total_conversions/ df_an.cumsum_total_impressions
df_an['cumsumCPA'] = df_an.cumsum_total_cost/ df_an.cumsum_total_conversions
df_an['cumsumROAS'] = df_an.cumsum_total_revenue / df_an.cumsum_total_cost

#### Plot 6 - Share of clicks, conversions and revenue and % of cumulative cost vs frequency bands

In [23]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["reach"]/df_an["cumsum_reach"].max()*100,
                     name="% Reach", marker_color='#ff007f'), 
                     secondary_y=False)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_conversions"]/df_an["cumsum_total_conversions"].max()*100,
                     name="% Conversions", marker_color='#005daa'),
                     secondary_y=False,)

fig.add_trace(go.Bar(x=df_an.frequency_range,
                     y=df_an["total_revenue"]/df_an["cumsum_total_revenue"].max()*100,
                     name="% Revenue", marker_color='#a87532'),
                     secondary_y=False,)

fig.add_trace(go.Scatter(x=df_an.frequency_range, 
                         y=df_an.cumsum_total_cost/df_an.cumsum_total_cost.max()*100,
                         mode='lines+markers',
                         name='% of Total cost', marker_color='#415a6c'),
                         secondary_y=True,)


fig.update_layout(
    title='<b>Share of clicks, conversions and revenue and % of cumulative cost vs frequency bands</b>',
    #template="plotly_dark",
    xaxis = dict(
        tickmode = 'array',
        ),
    xaxis_title="<b>Frequency</b>",
    #xaxis_range=[0,10]    
)


fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0
))

# Set y-axes titles
fig.update_yaxes(title_text="<b>Percentage</b>", ticksuffix = "%", secondary_y=False)
fig.update_yaxes(title_text="<b>Cost</b>", ticksuffix = "%", secondary_y=True)

#### Tables - Frequency bands 5-10-15-20

In [35]:
df_an = df[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]
df_an = df_an[df_an.frequency > 0]

# Upper limit of each group
upper_limits = list(range(5,df.frequency.max(),5))

# Create a column called 'frequency_range' to group frequencies based on limits and one numerical column to sort values
df_an.loc[df_an['frequency']<=upper_limits[0],'frequency_range'] = f'1 - {upper_limits[0]}'
df_an.loc[df_an['frequency']<=upper_limits[0],'order'] = 0
for index,value in enumerate(upper_limits[:-1]):
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'frequency_range'] = f'{upper_limits[index]+1} - {upper_limits[index+1]}'
    df_an.loc[(df_an['frequency']>upper_limits[index])&(df_an['frequency']<=upper_limits[index+1]),'order'] = index+1
df_an.loc[df_an['frequency']>upper_limits[-1],'frequency_range'] = f'>{upper_limits[-1]}'
df_an.loc[df_an['frequency']>upper_limits[-1],'order'] = len(upper_limits)+1
# display updated DataFrame
#df_an = df_an[['frequency', 'reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']].groupby(by='frequency_range').sum()
df_an = df_an.groupby(by=['order','frequency_range']).sum().reset_index()[['frequency_range','reach','total_impressions', 'total_clicks', 'total_conversions','total_revenue', 'total_cost']]

# Calculate cumulative variables
dimensions = [  
            'reach',
            'total_impressions',
            'total_clicks',
            'total_conversions',
            'total_revenue',
            'total_cost'
            ]
# For that dimension calculate cumsum and add it as a new column to the dataframe
for column in dimensions:
    df_an['cumsum_'+column] = df_an[column].cumsum()

# Calculate ROAS and CPA
df_an['CPA'] = df_an.cumsum_total_cost/ df_an.cumsum_total_conversions
df_an['ROAS'] = df_an.cumsum_total_revenue / df_an.cumsum_total_cost

df_cumsum = df_an[['frequency_range','cumsum_reach','cumsum_total_impressions', 'cumsum_total_clicks','cumsum_total_conversions', 'cumsum_total_revenue','cumsum_total_cost','CPA','ROAS']]
df_cumsum.to_clipboard()

In [36]:
df_cumsum2 = df_cumsum.copy(deep=True)
df_cumsum2[['cumsum_reach','cumsum_total_impressions', 'cumsum_total_clicks','cumsum_total_conversions', 'cumsum_total_revenue','cumsum_total_cost']] = df_cumsum2[['cumsum_reach','cumsum_total_impressions', 'cumsum_total_clicks','cumsum_total_conversions', 'cumsum_total_revenue','cumsum_total_cost']].apply(lambda x:100*x/max(x))
df_cumsum2.to_clipboard()